In [ ]:
import pandas as pd

In [ ]:
# Read in the CSVs needed to make the join.
ticker = pd.read_csv('data/asx-tickers.csv')
headlines = pd.read_csv('data/abcnews-date-text.csv')

In [ ]:
ticker.head(2)

In [ ]:
headlines.head(2)

In [ ]:
import numpy as np
from multiprocessing import cpu_count, Pool
 
cores = cpu_count()
 
def parallelize(data, func):
    data_split = np.array_split(data, cores)
    pool = Pool(cores)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

In [ ]:
from fuzzywuzzy import process, fuzz

# Fuzzy match function
def fuzzy_match(x, choices, scorer, cutoff):
    return process.extractOne(
        x, choices=choices, scorer=scorer, score_cutoff=cutoff
    )

# Parallelization function, to run on each split of data
def appfun(df):
    return df.loc[:, 'headline_text'].apply(
        fuzzy_match,
        args=(
            ticker.loc[:, 'company'], 
            fuzz.token_set_ratio,
            80
        )
    )

# Run in parallel
matching_results = parallelize(headlines, appfun)